In [ ]:
import pandas as pd
import re
import datetime

## Loading and Displaying initial data

In [ ]:
df = pd.read_csv('../Extract/amazon_0507.csv')
df.head(2)

In [ ]:
df.info()

## Defining functions for transformation and cleaning

In [ ]:
def del_na(column):
    return column[column.notna()]

In [ ]:
def only_first(cell, cast='int'):
    if cast == 'int':
        return int(cell.split()[0])
    else:
        return float(cell.split()[0])

In [ ]:
def to_watts(cell):
    cell = cell.split(' ')
    watts = float(cell[0])
    if 'kilowatt' in cell[1]:
        watts *=  1000
    elif 'milliamp' in cell[1]:
        volts = 12
        watts = (watts*volts)/1000
    
    return int(watts)

In [ ]:
def cast_int(value):
    try:
        return int(value)
    except:
        return None        

In [ ]:
def rename_col(dict_name):
    df.rename(columns = dict_name, inplace = True)

In [ ]:
months_list = ["enero", "febrero", "marzo", 
                "abril", "mayo", "junio", 
                "julio", "agosto", "septiembre", 
                "octubre", "noviembre", "diciembre"]
month_dict = {}
month_value = 1

for month in months_list:
    month_dict[month] = month_value
    month_value += 1

def to_date(date_text):
    date_text = date_text.split()
    day = int(date_text[0])
    month = month_dict[date_text[2]]
    year = int(date_text[4])

    return datetime.date(year, month, day)

## Transforming data

In [ ]:
# Deleted empty column
df.drop(df.columns[35], axis='columns', inplace=True)

In [ ]:
# Delete '/n' of each name
space = re.compile('^\s+\n*(.*?.)\s+\n\s*')

df.name = df.name.apply( 
    lambda x: space.sub(r'\1', x) )

In [ ]:
# price_current
df.price_current = del_na(df.price_current).apply(
    lambda x: float( x.replace('$', '').replace(',', '')) )

In [ ]:
# price_original
df.price_original = del_na(df.price_original).apply(
    lambda x: float(x.replace('$', '').replace(',', '')) )

In [ ]:
# reviews_number
df.reviews_number = del_na(df.reviews_number).apply(
    lambda x: only_first(x.replace(',', ''), 'int') )

In [ ]:
# score
df.score = del_na(df.score).apply(
    only_first, args = ('float',) )

In [ ]:
# alto
rename_col({'Alto del producto' : 'height_cm'})

df.height_cm = del_na(df.height_cm).apply( 
    lambda x:  only_first(x, 'float')   if 'cent' in x    else only_first(x) / 100 )

In [ ]:
# ancho
rename_col({'Ancho del producto' : 'width_cm'})

df.width_cm = del_na(df.width_cm).apply( 
    lambda x: only_first(x, 'float')    if 'cent' in x    else only_first(x) / 100 )

In [ ]:
# screen_size_in
rename_col({'Tamaño de la pantalla' : 'screen_size_in'})
df.screen_size_in = del_na(df.screen_size_in).apply(
    lambda x: round(only_first(x, 'float'), 0))
df.screen_size_in = df.screen_size_in.astype('category')

In [ ]:
# proc_speed_ghz
rename_col({'Velocidad del procesador' : 'proc_speed_ghz'})

df.proc_speed_ghz = del_na(df.proc_speed_ghz).apply(only_first, args = ('float',) )

In [ ]:
# ram
rename_col({'Tamaño de RAM': 'ram'})

df.ram = del_na(df.ram).apply(only_first)

In [ ]:
# ram_max
rename_col({'Memoria máxima compatible': 'ram_max'})

df.ram_max = del_na(df.ram_max).apply(only_first, args=('float',) )

In [ ]:
# memory
rename_col({'Tamaño de la unidad de disco duro' : 'memory',})

df.memory = del_na(df.memory).apply(only_first)

In [ ]:
# gpu_ram
rename_col({'Tamaño de RAM de la tarjeta gráfica' : 'gpu_ram',})

df.gpu_ram = del_na(df.gpu_ram).apply(only_first)

In [ ]:
# usb_2
rename_col({'Número de puertos USB 2.0': 'usb_2',})

df.usb_2 = del_na(df.usb_2).apply(int)

In [ ]:
# usb_3
rename_col({'Número de puertos USB 3.0': 'usb_3',})

df.usb_3 = del_na(df.usb_3).apply(int)

In [ ]:
# battey
rename_col({'Lithium Battery Energy Content' : 'battery_wh',})

df.battery_wh = del_na(df.battery_wh).apply(to_watts)

In [ ]:
# battery_cells
rename_col({'Number of Lithium Ion Cells' : 'battery_cells',})
df.battery_cells = del_na(df.battery_cells).apply(int)
df.battery_cells = df.battery_cells.astype('category')

In [ ]:
# weight_kg
rename_col({'Peso del envío' : 'weight_kg'})

df.weight_kg = del_na(df.weight_kg).apply(only_first, args=('float',) )

In [ ]:
# new column with resolution X of scree from screen resolution
df['resolution_x'] = del_na(df['Resolución de la pantalla']).apply(
    lambda x: x.replace(' ', '').split('x')[0]
    ).apply(cast_int)

In [ ]:
# new column with resolution Y of screen from screen resolution
df['resolution_y'] = del_na(df['Resolución de la pantalla']).apply( 
    lambda x: x.split('x')[1].split()[0] if len(x.split('x')) > 1 else None 
    ).apply(cast_int)

In [ ]:
# Ordering columns ['resolution_x' & 'resolution_y'] after column index 12
df = df[df.columns.to_list()[:13] + ['resolution_x', 'resolution_y'] + df.columns.to_list()[12:-2]]

In [ ]:
# Drop old column of screen resolution
del df['Resolución de la pantalla']

In [ ]:
rename_col({'Tipo de memoria del equipo' : 'ram_type'})
df.ram_type = del_na(df.ram_type).apply(lambda x: x.split()[0])
df.ram_type = df.ram_type.astype('category')

In [ ]:
# since
rename_col({'Producto en Amazon.com.mx desde' : 'since'})
df.since = df.since.apply(to_date)
df.since = df.since.astype('datetime64[ns]')

## Transforming DTypes

In [ ]:
# Perform types of DataFrame
df = df.convert_dtypes()


In [ ]:
rename_col({'Marca' : 'brand'})
df.brand = df.brand.astype('category')

In [ ]:
df.seller = df.seller.astype('category')

In [ ]:
rename_col({'Marca del procesador' : 'proc_brand'})
df.proc_brand = df.proc_brand.astype('category')

In [ ]:
rename_col({'Interfaz de la unidad de disco duro' : 'memory_interface'})
df.memory_interface = df.memory_interface.astype('category')

In [ ]:
rename_col({'Sistema operativo' : 'os'})
df.os = df.os.astype('category')

In [ ]:
rename_col({'Interfaz de la tarjeta gráfica' : 'gpu_interface'})
df.gpu_interface = df.gpu_interface.astype('category')

## Showing clean data and exporting processed data

In [ ]:
df.info()

In [ ]:
df.head(2)

In [ ]:
df.to_csv('amazon_clean_' + datetime.datetime.today().strftime('%m%d') + '.csv')

In [ ]:
df_filtered = df.drop(columns=[
    'Tecnología de la memoria', 
    'Tipo de unidad óptica', 
    'Tipo de conexión inalámbrica', 
    'Número de modelo del producto',
    'usb_3',
    'usb_2',
    'Series',
    'Color',
    'Descripción del disco duro',
    'Tipo de RAM para gráficos',
    'memory_interface',
    'gpu_interface',
    'resolution_x',
    'resolution_y',
    'gpu_ram',
    'Descripción de la tarjeta gráfica',
    'Coprocesador de gráficos',
    'Tipo de procesador'
    ])

In [ ]:
df.info()